In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 24 2025

@author: Yaning
"""

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModelWithLMHead
import csv
import Levenshtein

In [ ]:
model_name = "dbmdz/german-gpt2"  # Use the German GPT-2 model
tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")
model = AutoModelWithLMHead.from_pretrained(model_name)

/home/yahu202d/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yahu202d/.local/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:1833: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [11]:
saechsisch_to_hochdeutsch = {
    "Guhdn Dach": "Guten Tag",
    "Wie jeht's'n?": "Wie geht es dir?",
    "Nu": "Ja",
    "Nee": "Nein",
    "Isch": "Ich",
    "Deh": "Du",
    "Mir": "Wir",
    "Hiehr": "Hier",
    "Dor": "Da",
    "Isch geh orbeed'n": "Ich gehe zur Arbeit",
    "Scheen": "Schön",
    "Mächen": "Mädchen",
    "Bursch": "Junge",
    "Isch liib di": "Ich liebe dich",
    "Grombern": "Kartoffeln",
    "Keek": "Kuchen",
    "S'schnet": "Es regnet",
    "Des is mir eejal.": "Das ist mir egal.",
    "Maach lamm!": "Mach langsam!",
    "Was machste?": "Was machst du?",
    "Isch hab keen Bock.": "Ich habe keine Lust.",
    "Des isch aba gudd!": "Das ist aber gut!",
    "Des wor schenn witzich.": "Das war lustig.",
    "Bemme": "Brot",
    "Schnaddel": "Mund",
    "Fläschl": "Flasche",
    "Schdriewel": "Strümpfe",
    "Äppel": "Äpfel",
    "Dräsdn": "Dresden",
    "Schdöckl": "Stock",
    "Schdowwel": "Schaufel",
    "Friedersdor": "Friedersdorf",
    "Budde": "Dose",
    "Hauschdier": "Haustier",
    "De Brogge": "Die Brücke",
    "Schäppel": "Kanne",
    "Häusl": "Haus",
    "Fechdner": "Fischer",
    "Schdelln": "Stellen",
    "De Schdroß": "Die Straße",
    "Schdahln": "Stehlen",
    "De Scheenheit": "Die Schönheit",
    "Hasde?": "Hast du?",
    "Nischd": "Nichts",
    "Schdrieh": "Strich",
    "Bleede G'schicht": "Blöde Geschichte",
    "Denkste?": "Denkst du?",
    "Sich dräschd'n": "Sich drehen",
    "Schdowhl": "Stuhl",
    "Grubbsch": "Hand",
    "Scharrn": "Scharren",
    "Schdiebn": "Schieben",
    "Guggsch!": "Siehst du!",
    "Uffm": "Auf dem",
    "Huscht'r": "Hustet er",
    "Schlumpe": "Schlampe",
    "Karrnsch": "Karre",
    "Gugg ma do!": "Schau mal da!",
}

In [6]:
def calculate_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
    return torch.exp(loss).item()


def find_closest_word(word, reference_words):
    similarities = [(ref, Levenshtein.distance(word, ref)) for ref in reference_words]
    closest_word = min(similarities, key=lambda x: x[1])  # Find word with minimum distance
    return closest_word

def replace_words_in_text(texts, dictionary, keys):
    updated_texts = []
    for text in texts:
        words = text.split()
        updated_words = []
        for word in words:
            if word in keys:
                # Replace the word with the dictionary value (Hochdeutsch)
                updated_words.append(dictionary[word])
            else:
                # Keep the word as it is if it's not in the dictionary
                updated_words.append(word)
        updated_texts.append(" ".join(updated_words))
    return updated_texts

Perplexity: 58.210426330566406


In [2]:
# load the dataset
with open('/home/yahu202d/workspaces/horse/yahu202d-saexy/extracted.csv', mode='r', newline='') as file:
    reader = csv.reader(file)
    
    for r in reader:
        texts = r

texts = texts[:3]

In [3]:
texts

['Albrechtsburg und Dom zu Meißen Ihr geent mirsch globn ich war schon iweralltIch war in Bayernlande un in BreißenIch hab an Rheine Borgen abgemaltDoch egal ziehts mich widder hin nach MeißenDas liegt so scheen un was gibts da for WeinDer is Sie nich so heemdiksch wie an Rhein Ideale Gebirgslandschaft mit Wasserfall Zeichnung von Caspar David Friedrich Un gommt mer nu ins hibsche scheene DräsenVerdimian sis wie in BaradiesWer da noch niche Gindersch is gewesenMit den sein Genntnissen Gottschtrambach is es miesDe säksche Schweiz is scheenste WeltgebärgeMer gommt sich drinne vor wie in der Gärche Un wolltr äwas sähn von richtgen LäbnDa mißter Eich ämal nach Leipzg bemiehnDagegen sin die andern Städtchen äbnWie dach un Nacht gorzum der reene GiehnDe Mächen außerdem so sieß un liebentbranntNee bist Du scheen mei deires Saksenland',
 'Du schiene Schwammezeitdu bist de schännste LustMr hot oft Glück aa kaansun wird darb ausgerustWersch kennt dan läßts kaa Ruhmacht naus eh der Tag abrichtDe 

In [7]:
unknown_words = {}
for text in texts:
    words = text.split()
    for word in words:
        word_perplexity = calculate_perplexity(model, tokenizer, word)
        unknown_words[word] = word_perplexity

In [8]:
unknown_words

{'Albrechtsburg': 15.386807441711426,
 'und': nan,
 'Dom': nan,
 'zu': nan,
 'Meißen': 7309.2392578125,
 'Ihr': nan,
 'geent': 11598.412109375,
 'mirsch': 3061.35302734375,
 'globn': 2410.3818359375,
 'ich': nan,
 'war': nan,
 'schon': nan,
 'iweralltIch': 45775.02734375,
 'in': nan,
 'Bayernlande': 303491.78125,
 'un': nan,
 'BreißenIch': 2444874.0,
 'hab': nan,
 'an': nan,
 'Rheine': 94.63468170166016,
 'Borgen': 5239.79345703125,
 'abgemaltDoch': 25456.4609375,
 'egal': nan,
 'ziehts': 2907.605712890625,
 'mich': nan,
 'widder': 1933.6171875,
 'hin': nan,
 'nach': nan,
 'MeißenDas': 33375.16796875,
 'liegt': nan,
 'so': nan,
 'scheen': 1080.65380859375,
 'was': nan,
 'gibts': 84.81922149658203,
 'da': nan,
 'for': nan,
 'WeinDer': 1523473.625,
 'is': nan,
 'Sie': nan,
 'nich': 1565.0115966796875,
 'heemdiksch': 8648.826171875,
 'wie': nan,
 'Rhein': nan,
 'Ideale': 22.294404983520508,
 'Gebirgslandschaft': 33.099979400634766,
 'mit': nan,
 'Wasserfall': 1732.5189208984375,
 'Zeichnu

In [9]:
perplexity_threshold = 1000.0

# Filter unknown words
unknown_words_filtered = {word: p for word, p in unknown_words.items() if p > perplexity_threshold}

print("Unknown Words:", unknown_words_filtered)

Unknown Words: {'Meißen': 7309.2392578125, 'geent': 11598.412109375, 'mirsch': 3061.35302734375, 'globn': 2410.3818359375, 'iweralltIch': 45775.02734375, 'Bayernlande': 303491.78125, 'BreißenIch': 2444874.0, 'Borgen': 5239.79345703125, 'abgemaltDoch': 25456.4609375, 'ziehts': 2907.605712890625, 'widder': 1933.6171875, 'MeißenDas': 33375.16796875, 'scheen': 1080.65380859375, 'WeinDer': 1523473.625, 'nich': 1565.0115966796875, 'heemdiksch': 8648.826171875, 'Wasserfall': 1732.5189208984375, 'hibsche': 120850.046875, 'scheene': 20019.552734375, 'DräsenVerdimian': 22176.580078125, 'BaradiesWer': 196891.8125, 'niche': 33228.62890625, 'Gindersch': 3117.835205078125, 'gewesenMit': 8782.2861328125, 'Genntnissen': 2347593.75, 'Gottschtrambach': 7233.1630859375, 'miesDe': 29559.173828125, 'säksche': 2413.096923828125, 'scheenste': 1861.3343505859375, 'WeltgebärgeMer': 204731.375, 'drinne': 1407.9088134765625, 'Gärche': 2081.264404296875, 'wolltr': 4811.9287109375, 'äwas': 62066.52734375, 'sähn': 

In [10]:
len(unknown_words_filtered)

133

In [13]:
saechsisch_keys = list(saechsisch_to_hochdeutsch.keys())

In [20]:
poem_dicitonary = {}

In [26]:
for key in unknown_words_filtered.keys():
    closest_sachsen = find_closest_word(key, saechsisch_keys)
    translation = saechsisch_to_hochdeutsch[closest_sachsen[0]]
    print(closest_sachsen, translation)
    poem_dicitonary[key] = translation


('Mächen', 3) Mädchen
('Nee', 3) Nein
('Bursch', 2) Junge
('Dor', 4) Da
('Bursch', 8) Junge
('Maach lamm!', 8) Mach langsam!
('Bursch', 7) Junge
('Dor', 4) Da
('Guhdn Dach', 9) Guten Tag
('Hiehr', 3) Hier
('Budde', 3) Dose
('Mächen', 6) Mädchen
('Scheen', 1) Schön
('Nee', 5) Nein
('Isch', 2) Ich
('Isch', 7) Ich
('Hasde?', 7) Hast du?
('Nischd', 3) Nichts
('Scheen', 2) Schön
('Dräsdn', 9) Dresden
('Hiehr', 8) Hier
('Isch', 3) Ich
('Bursch', 5) Junge
('Scheen', 7) Schön
('Denkste?', 7) Denkst du?
('Guhdn Dach', 10) Guten Tag
('Nee', 4) Nein
('Isch', 4) Ich
('Scheen', 4) Schön
('Schdriewel', 11) Strümpfe
('Nee', 5) Nein
('Mächen', 3) Mädchen
('Dor', 4) Da
('Nu', 4) Ja
('Isch', 3) Ich
('Scheen', 4) Schön
('Mächen', 5) Mädchen
('Mir', 4) Wir
('Isch', 2) Ich
('Häusl', 3) Haus
('Nee', 5) Nein
('Scheen', 10) Schön
('Nee', 5) Nein
('Dor', 4) Da
('Nee', 3) Nein
('Hiehr', 4) Hier
('Mächen', 0) Mädchen
("Wie jeht's'n?", 12) Wie geht es dir?
('Nee', 4) Nein
('Schdelln', 6) Stellen
('Scheen', 3) Sch

In [28]:
poem_dictionary_keys = list(poem_dicitonary.keys())

In [30]:
replace_words_in_text(texts, poem_dicitonary, poem_dictionary_keys)


['Fischer Ja Da Ja Mädchen Wir Nein Junge Da Ich Wir Ich Junge Wir Ja Mach langsam! Ja Ja Junge Ja Ja Nein Da Guten Tag Ja Hier Ich Dose Wir Ich Mädchen Hier Ja Schön Ja Ja Wir Ja Da Nein Ja Nein Ich Ja Ich Nein Ja Schön Nein Hand Wir Hast du? Schön Da Hast du? Du Friedersdorf Ja Brot Nein Ja Ja Nichts Schön Dresden Wir Nein Ja Hier Ja Ich Ich Junge Ja Schön Nein Nein Denkst du? Guten Tag Ja Ja Nein Ich Schön Ja Schön Strümpfe Brot Ich Nein Da Nein Ja Nein Mädchen Ja Da Ja Ich Da Schön Mädchen Wir Ich Haus Ich Nein Schön Wir Nein Schön Mädchen Nein Ich Ja Ich Da Nein Nein Hier Mädchen Dose Ja Nein Ja Wie geht es dir? Wir Ja Schön Nein Nein Stellen',
 'Ja Schön Schön Wir Ja Schön Hustet er Da Ja Ich Ja Dresden Wir Wir Junge Nein Ja Haus Ja Junge Ja Du Nein Ja Junge Schön Brot Ich Ja Junge Du Ich liebe dich Nein Mund Auf dem Nein Hier Wir Hier Ja Was machst du? Flasche Ja Nein Strümpfe Kanne Ja Karre Ja Ich Nein Ja Kuchen Nein Ja Ja Hier Ich Schlampe Hier Ich Ja Ja Kuchen Ja Ja Ja Hier F